## What I Learned About Time Series Analysis

Looking at `timeseries.py, I got the following categorical_crossentropy loss for each epoch (math.floor): 
* MLP:  2180.558 → 1.791 → 1.791 → 1.790 → 1.790 → 1.790 → 1.789 → 1.789 → 1.789 → 1.789
* CNN: 5.060 → 1.791 → 1.790 → 1.790 → 1.790 → 1.789 → 1.789 → 1.789 → 1.789 → 1.789 

Both MLP and CNN start with very high loss (675 / 37) that quickly collapses to around 1.7, then stays flat.

When you look closer you can see that these models are randomly guessing. Where

$$
loss \approx -\log{\frac{1}{N}}
$$ 

N is the number of classes. Look at the code below

With only one layer of LSTM, does slightly better than random guessing (but not strictly better): 
* 1.820 → 1.788 → 1.788 → 1.799 → 1.791 → 1.786 → 1.800 → 1.788 → 1.782 → 1.778 

In [20]:
import math
import os
import numpy as np
import keras

In [5]:
N = 6
- math.log(1/N)

1.791759469228055

In [6]:
fname = os.path.join("jena_climate_2009_2016.csv")

with open(fname) as f: 
    data = f.read()
    lines = data.split("\n") 
    header = lines[0].split(",") 
    lines = lines[1:] 
    print(header) 
    print(len(lines))

['"Date Time"', '"p (mbar)"', '"T (degC)"', '"Tpot (K)"', '"Tdew (degC)"', '"rh (%)"', '"VPmax (mbar)"', '"VPact (mbar)"', '"VPdef (mbar)"', '"sh (g/kg)"', '"H2OC (mmol/mol)"', '"rho (g/m**3)"', '"wv (m/s)"', '"max. wv (m/s)"', '"wd (deg)"']
420451


In [10]:
temperature = np.zeros((len(lines),))
raw_data = np.zeros((len(lines), len(header) - 1)) 
raw_data.shape


(420451, 14)

In [14]:
num_train_samples = int(0.5 * len(raw_data))
num_val_samples = int(0.25 * len(raw_data))
num_test_samples = len(raw_data) - num_train_samples - num_val_samples

In [19]:
mean = raw_data[:num_train_samples].mean(axis=0)
raw_data -= mean
std = raw_data[:num_train_samples].std(axis=0)
raw_data /= std
raw_data.shape

(420451, 14)

In [21]:
sampling_rate = 6
sequence_length = 120
delay = sampling_rate * (sequence_length + 24 - 1)
batch_size = 256

In [22]:
train_dataset = keras.utils.timeseries_dataset_from_array( 
    raw_data[:-delay],
    targets=temperature[delay:], 
    sampling_rate=sampling_rate, 
    sequence_length=sequence_length,
    shuffle=True,
    batch_size=batch_size,
    start_index=0,
    end_index=num_train_samples)

val_dataset = keras.utils.timeseries_dataset_from_array( 
    raw_data[:-delay],
    targets=temperature[delay:], 
    sampling_rate=sampling_rate, 
    sequence_length=sequence_length,
    shuffle=True,
    batch_size=batch_size,
    start_index=num_train_samples,
    end_index=num_train_samples + num_val_samples)

test_dataset = keras.utils.timeseries_dataset_from_array( 
    raw_data[:-delay],
    targets=temperature[delay:], 
    sampling_rate=sampling_rate, 
    sequence_length=sequence_length,
    shuffle=True,
    batch_size=batch_size, 
    start_index=num_train_samples + num_val_samples) 

for samples, targets in train_dataset:
    print("samples shape:", samples.shape)
    print("targets shape:", targets.shape)
    break

samples shape: (256, 120, 14)
targets shape: (256,)


In [ ]:
inputs = keras.Input(shape=(sequence_length, raw_data.shape[-1])) 